In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = xr.open_dataset('data/wind_temperature.nc')
data

<xarray.Dataset>
Dimensions:    (longitude: 3, latitude: 3, time: 8760)
Coordinates:
  * longitude  (longitude) float32 76.8 77.05 77.3
  * latitude   (latitude) float32 28.9 28.65 28.4
  * time       (time) datetime64[ns] 2022-01-01 ... 2022-12-31T23:00:00
Data variables:
    t          (time, latitude, longitude) float32 ...
    u          (time, latitude, longitude) float32 ...
    v          (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-02 20:16:03 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [2]:
df = data.to_dataframe().reset_index()
df

,longitude,latitude,time,t,u,v
0,76.800003,28.9,2022-01-01 00:00:00,248.909729,-9.257841,-28.587065
1,76.800003,28.9,2022-01-01 01:00:00,249.046753,-8.018175,-27.149031
2,76.800003,28.9,2022-01-01 02:00:00,247.822647,-8.475103,-23.520796
3,76.800003,28.9,2022-01-01 03:00:00,247.266708,-8.973751,-18.165319
4,76.800003,28.9,2022-01-01 04:00:00,247.585617,-9.007524,-14.171290
...,...,...,...,...,...,...
78835,77.300003,28.4,2022-12-31 19:00:00,260.457428,42.351227,16.476660
78836,77.300003,28.4,2022-12-31 20:00:00,259.598694,43.207474,15.315258
78837,77.300003,28.4,2022-12-31 21:00:00,258.214752,42.861794,16.586397
78838,77.300003,28.4,2022-12-31 22:00:00,258.606506,36.017803,17.696358


In [3]:
uniq = df.drop_duplicates(subset = ['latitude', 'longitude'])[['latitude', 'longitude']]

In [4]:
uniq

,latitude,longitude
0,28.90,76.800003
8760,28.65,76.800003
17520,28.40,76.800003
26280,28.90,77.050003
35040,28.65,77.050003
43800,28.40,77.050003
52560,28.90,77.300003
61320,28.65,77.300003
70080,28.40,77.300003


In [5]:
final_df = pd.DataFrame()

# Method 1: Interpolation

In [6]:
for index, row in uniq.iterrows():
    d = df[(df['latitude'] == row['latitude']) & (df['longitude'] == row['longitude'])]
    
    d = d.set_index(d['time'])
    d = d.resample('30min').asfreq()
    
    d.loc['2022-12-31 23:30:00'] = {'latitude': 28.4, 'longitude': 77.300003}
    d['latitude'] = d['latitude'].fillna(method = 'bfill')
    d['longitude'] = d['longitude'].fillna(method = 'bfill')
    d['t'] = d['t'].interpolate(method = 'linear')
    d['u'] = d['u'].interpolate(method = 'linear')
    d['v'] = d['v'].interpolate(method = 'linear')
    
    d = d.drop(columns = ['time'])
    d.reset_index()
    
    final_df = pd.concat([final_df, d])

In [7]:
final_df = final_df.reset_index()

In [8]:
final_df['time'] = pd.to_datetime(final_df['time'])

In [9]:
final_df

,time,longitude,latitude,t,u,v
0,2022-01-01 00:00:00,76.800003,28.9,248.909729,-9.257841,-28.587065
1,2022-01-01 00:30:00,76.800003,28.9,248.978241,-8.638008,-27.868048
2,2022-01-01 01:00:00,76.800003,28.9,249.046753,-8.018175,-27.149031
3,2022-01-01 01:30:00,76.800003,28.9,248.434692,-8.246639,-25.334913
4,2022-01-01 02:00:00,76.800003,28.9,247.822647,-8.475103,-23.520796
...,...,...,...,...,...,...
157675,2022-12-31 21:30:00,77.300003,28.4,258.410645,39.439796,17.141376
157676,2022-12-31 22:00:00,77.300003,28.4,258.606506,36.017803,17.696358
157677,2022-12-31 22:30:00,77.300003,28.4,258.477753,35.680073,18.425091
157678,2022-12-31 23:00:00,77.300003,28.4,258.348999,35.342342,19.153826


In [10]:
final_df.to_csv('data\wind_temp_interpolated.csv')

# Method 2: Shift data by 30 mins

In [11]:
df['time'] = pd.to_datetime(df['time'])

In [12]:
from datetime import timedelta
def shift_time(x):
    x['time'] = x['time'] + timedelta(minutes = 30)
    return x

df = df.apply(lambda x: shift_time(x), axis = 1)

In [13]:
df

,longitude,latitude,time,t,u,v
0,76.800003,28.9,2022-01-01 00:30:00,248.909729,-9.257841,-28.587065
1,76.800003,28.9,2022-01-01 01:30:00,249.046753,-8.018175,-27.149031
2,76.800003,28.9,2022-01-01 02:30:00,247.822647,-8.475103,-23.520796
3,76.800003,28.9,2022-01-01 03:30:00,247.266708,-8.973751,-18.165319
4,76.800003,28.9,2022-01-01 04:30:00,247.585617,-9.007524,-14.171290
...,...,...,...,...,...,...
78835,77.300003,28.4,2022-12-31 19:30:00,260.457428,42.351227,16.476660
78836,77.300003,28.4,2022-12-31 20:30:00,259.598694,43.207474,15.315258
78837,77.300003,28.4,2022-12-31 21:30:00,258.214752,42.861794,16.586397
78838,77.300003,28.4,2022-12-31 22:30:00,258.606506,36.017803,17.696358


In [14]:
df.to_csv('data/wind_temp_shifted.csv')